# AI Agent

This notebook shows how to create AI agent, which can answer to questions without LangChain, HayStack or other libraries.
This agent will be able to answer to questions using different tools, like Wikipedia, Calculator or other.

It uses FSM (using [transitions](https://github.com/pytransitions/transitions) library) to implement agent's behaviour and huggingface's [transformers](https://huggingface.co/transformers/) library to work with LLMs.

## Installation

First of all, we need to install all required libraries.

In [ ]:
!pip install transitions transformers accelerate

## Initialization of model

We will use [transformers](https://huggingface.co/transformers/) library to work with T5 model.

Nothing special here, just initialization of model.

In [ ]:
from transformers import pipeline

model = "google/flan-t5-xxl"

pipe = pipeline("text2text-generation", model=model, device_map="auto", model_kwargs={"load_in_8bit": True})


In [ ]:
# Check if model works correctly for our task
test_result = pipe("Answer only Yes/No. Honestly. Can you answer to this question: 'Is there any meaning of life'. Answer: ")
assert test_result[0]["generated_text"].strip() == "No", "Model doesn't work correctly"

## Agent

This class implements agent, which can answer to questions. It has following states:
- waiting_for_task
    It is initial state. Agent is waiting for request.
- choosing_tool
    Agent is choosing tool to answer the question.
- applying_tool
    Agent is applying tool to answer the question.
- finished
    Agent is answering to the question.

This class isn't perfect, but it is good enough to show the idea.
It uses Finite State Machine to implement agent's behaviour. It is implemented using [transitions](https://github.com/pytransitions/transitions) library.

In [ ]:
from typing import Dict, Callable, List, Union
import logging
from transitions import Machine


class Agent(object):
    states = [
        'waiting_for_task',
        'choosing_tool',
        'applying_tool',
        'finished'
    ]

    transitions = [
        {
            "source": "waiting_for_task",
            "trigger": "request",
            "dest": "choosing_tool",
        },
        {
            "source": "choosing_tool",
            "trigger": "apply_tool",
            "dest": "applying_tool",
        },
        {
            "source": "choosing_tool",
            "trigger": "answer",
            "dest": "finished",
        },
        {
            "source": "applying_tool",
            "trigger": "answer",
            "dest": "finished",
        }
    ]

    prompt_fn: Callable = None
    tools: List[Dict[str, Union[str, Callable[[str], str]]]] = None
    machine: Machine = None
    final_answer: str = None

    def __init__(self, prompt_fn: Callable, tools: List[Dict[str, Union[str, Callable[[str], str]]]]):
        self.prompt_fn = prompt_fn
        self.tools = tools
        self.machine = Machine(
            model=self,
            states=self.states,
            transitions=self.transitions,
            initial='waiting_for_task',
            send_event=True
        )

    def on_enter_choosing_tool(self, event):
        print("Choosing tool")
        tools_list = "\n".join(["{}: {}".format(tool["name"], tool["description"]) for tool in self.tools])
        prompt = """
            You have following tools:
            {tools}
            To choose tool, type name only.
            If you don't need any tool to answer the questuin, type 'None'.
            Giver Request: {user_query}
        """.format(tools=tools_list, user_query=event.kwargs.get("user_query"))
        tool = self.prompt_fn(prompt)
        logging.log(logging.INFO, "Tool: {}".format(tool))
        if tool == "None":
            self.answer(user_query=event.kwargs.get("user_query"))
        else:
            self.apply_tool(tool=tool, user_query=event.kwargs.get("user_query"))

    def on_enter_applying_tool(self, event):
        print("Applying tool")
        tool = next(filter(lambda tool: tool["name"] == event.kwargs.get("tool"), self.tools))
        tool_result = tool["func"](event.kwargs.get("user_query"))
        logging.log(logging.INFO, "Tool result: {}".format(tool_result))
        self.answer(user_query=event.kwargs.get("user_query"), tool=event.kwargs.get("tool"), tool_result=tool_result)

    def on_enter_finished(self, event):
        print("Answering")
        prompt = """
            AI: I made investigation about your query:
            "{user_query}"
            AI: I used {tool} and it gave me following result:
            "{tool_result}"
            AI: So, I think the answer is:
        """.format(
            user_query=event.kwargs.get("user_query"),
            tool=event.kwargs.get("tool"),
            tool_result=event.kwargs.get("tool_result")
        ).strip()
        answer = self.prompt_fn(prompt).strip()
        logging.log(logging.INFO, "Prompt: {prompt} {answer}".format(prompt=prompt, answer=answer))
        self.final_answer = "{thoughts} {answer}".format(thoughts=prompt, answer=answer)


## Usage

Implementation took 92 lines of code. It's bulletproof, no bugs, no errors, no exceptions, no unexpected breaking interface changes. And it works with OSS models.

Let's try to use it.

First of all, we need to define some tools. We will use two tools:
- Wikipedia - TODO - Just a stub for now
- Google - TODO - Just a stub for now
- Calculator - TODO - Just a stub for now

Also we need to create wrapper for our model. It will be used to generate text. If you want to use another model, you need to change only this function.

Also i put some logging to see what's going on, sorry for noise.

In [ ]:
logging.basicConfig(level=logging.INFO)
logging.getLogger('transitions').setLevel(logging.INFO)


# And some transitions between states. We're lazy, so we'll leave out

def make_query(prompt: str) -> str:
    return pipe(prompt)[0]["generated_text"]


tools_dict: List[Dict[str, Union[str, Callable[[str], str]]]] = [
    {
        "name": "Wikipedia",
        "description": "Useful for general questions",
        "func": lambda
            query: "It is a very important person or object. It was created in 1990 by Tim Berners-Lee. And it is very useful."
    },
    {
        "name": "Google",
        "description": "Useful for actual events",
        "apply": lambda query: "It never happened"
    },
    {
        "name": "Calculator",
        "description": "Useful for complex calculations",
        "apply": lambda query: "It is 42"
    }
]

agent = Agent(
    prompt_fn=make_query,
    tools=tools_dict
)

print(agent.state)
agent.request(user_query="What is ms Yanda?")

## Future work

- Add real tools
- Add memory and it's propogation